### Assignment No. 2 - Supervised Learning

<img src="../docs/intro.webp" alt="titanic" style="width:700px;"/>

## Introduction

The RMS Titanic dataset is a classic example used in data science and machine learning to explore factors influencing survival rates during the tragic sinking of the Titanic. The objective is to build predictive models that can accurately determine whether a passenger survived or not based on various attributes such as socio-economic status, age, gender, and more.

In this supervised learning task, we aim to develop and compare machine learning algorithms to predict survival outcomes using historical passenger data. By leveraging techniques like decision trees, neural networks, and support vector machines, we will analyze and evaluate the effectiveness of these models in classifying passengers into survival categories.

Through this exercise, we seek to gain insights into the factors that contributed significantly to survival during this historical event, demonstrating the practical application of supervised learning techniques in understanding complex real-world scenarios.

[Titanic Problem Kaggle Page](https://www.kaggle.com/datasets/sakshisatre/titanic-dataset?resource=download)

## Importing libraries

We firstly need to install the libraries we will use in this project.

To do so, run the following command in the terminal (make sure you are in the project's root directory):

```pip install -r requirements.txt```

Then, we can import the libraries we will use in this project.

We also disable the warnings, to make the notebook cleaner.

In [ ]:
import warnings # Needed to ignore warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import math

warnings.filterwarnings('ignore')

## Create a dataframe with the dataset from the csv file

In [ ]:
df = pd.read_csv('../data/Cancer_Data.csv')

df.head()